In [ ]:
# για το scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# σύνδεση με το gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# βασικό url του site
main_url = 'https://www.kathimerini.gr/'
# αρχικό url των teaser pages
start_url = 'https://www.kathimerini.gr/society/page/'

start_page = 2
end_page = 50

In [ ]:
news_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  articles_list = doc.find('div', {'class': 'column p-0 category-roh'}).find_all('div', {'class': 'nx-article column is-full cat-template3'})


  for teaser in articles_list:
      story_dict = {}
      try:

        link = teaser.find('a', {'class': ''})['href']

        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      news_list.append(story_dict)

In [ ]:
kathimerini_df = pd.DataFrame(news_list)
kathimerini_df

,url
0,https://www.kathimerini.gr/society/562557544/d...
1,https://www.kathimerini.gr/society/562557586/p...
2,https://www.kathimerini.gr/society/562557565/t...
3,https://www.kathimerini.gr/society/562557361/i...
4,https://www.kathimerini.gr/society/562557505/e...
...,...
534,https://www.kathimerini.gr/society/562533760/p...
535,https://www.kathimerini.gr/society/562535656/k...
536,https://www.kathimerini.gr/society/562535638/o...
537,https://www.kathimerini.gr/society/562535587/f...


In [ ]:
full_articles_list = []

for article_url in kathimerini_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('article')
  full_article_dict = {}

  # website
  full_article_dict['site'] = "kathimerini.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'class': 'main-category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('div', {'class', "entry-content"}).find('span', {'class', 'author'}).text.replace('\n','')
    full_article_dict['Author'] = author
  except:
    full_article_dict['Author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  # media
  media = article.find('figure', {'class', 'image'}) or article.find('div', {'class', 'twitter-tweet'}) or article.find('div', {'class', 'html5-video-player'})
  if media:
    full_article_dict['Media'] = True
  else:
    full_article_dict['Media'] = False

  full_articles_list.append(full_article_dict)

kathimerini_df = pd.DataFrame(full_articles_list)
kathimerini_df

,site,url,section,title,date,Author,full_text,Media
0,newsbomb.gr,https://www.kathimerini.gr/society/562557544/d...,,Δήμαρχος Υδρας: «Τα ζώα εργασίας αποτελούν ανα...,2023-08-07T18:45:00+03:00,Newsroom,«Γίνεται 200 χρόνια στο νησί. Δεν παραβιάζεται...,False
1,newsbomb.gr,https://www.kathimerini.gr/society/562557586/p...,,Πύρινα μέτωπα σε Τρίπολη και Σέρρες,2023-08-07T18:10:04+03:00,Newsroom,Μάχη με τις φλόγες δίνουν επίγειες και εναέριε...,False
2,newsbomb.gr,https://www.kathimerini.gr/society/562557565/t...,,Το 23% της Αττικής κάηκε μέσα σε 7 χρόνια,2023-08-07T18:00:20+03:00,Newsroom,Τα περισσότερα στρέμματα κάηκαν το 2021 από τι...,False
3,newsbomb.gr,https://www.kathimerini.gr/society/562557361/i...,,Η μεγάλη διαδρομή του Ντέμπα από το Μάλι στη Φ...,2023-08-07T17:50:52+03:00,Ελβίρα Κρίθαρη,Η ιστορία του ασυνόδευτου πρόσφυγα από το Μάλι...,False
4,newsbomb.gr,https://www.kathimerini.gr/society/562557505/e...,,ΕΟΦ: Απαγόρευση διακίνησης συμπληρώματος διατρ...,2023-08-07T17:20:21+03:00,Newsroom,Ανιχνεύτηκε μη εγκεκριμένη ουσία Την απαγόρευσ...,False
...,...,...,...,...,...,...,...,...
534,newsbomb.gr,https://www.kathimerini.gr/society/562533760/p...,,,,,Αποκαλείται το «παράδοξο της φωτιάς»: Ο μηχανι...,True
535,newsbomb.gr,https://www.kathimerini.gr/society/562535656/k...,,Καιρός: Μικρή πτώση της θερμοκρασίας σήμερα – ...,2023-07-24T07:31:52+03:00,Newsroom,Η θερμοκρασία θα φτάσει σήμερα στα ηπειρωτικά ...,False
536,newsbomb.gr,https://www.kathimerini.gr/society/562535638/o...,,Ολονύκτια πολυμέτωπη μάχη με τις φλόγες,2023-07-24T06:57:52+03:00,Newsroom,Μάχη έδωσαν για ακόμη μια νύχτα δυνάμεις της Π...,True
537,newsbomb.gr,https://www.kathimerini.gr/society/562535587/f...,,Φωτιά στη Ρόδο: «Θα βοηθήσουμε και τον τελευτα...,2023-07-23T23:57:47+03:00,Λουκάς Βελιδάκης,Κάτοικοι του νησιού μιλούν στην «Κ» για το συγ...,False


In [ ]:
kathimerini_df.to_csv('drive/MyDrive/kathimerini.csv', index=False)